In [1]:
#importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression

import joblib

In [2]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

In [3]:
#load the data
file_name = 'TrainDataset2025.csv'
df = pd.read_csv(file_name)

# EDA 

In [4]:
#first 5 rows of the dataset
print(df.head())

          ID  pCR (outcome)  RelapseFreeSurvival (outcome)   Age  ER  PgR  \
0  TRG002174              1                          144.0  41.0   0    0   
1  TRG002178              0                          142.0  39.0   1    1   
2  TRG002204              1                          135.0  31.0   0    0   
3  TRG002206              0                           12.0  35.0   0    0   
4  TRG002210              0                          109.0  61.0   1    0   

   HER2  TrippleNegative  ChemoGrade  Proliferation  ...  \
0     0                1           3              3  ...   
1     0                0           3              3  ...   
2     0                1           2              1  ...   
3     0                1           3              3  ...   
4     0                0           2              1  ...   

   original_glszm_SmallAreaHighGrayLevelEmphasis  \
0                                       0.517172   
1                                       0.444391   
2                   

In [5]:
print(df.describe())

       pCR (outcome)  RelapseFreeSurvival (outcome)         Age          ER  \
count     400.000000                     400.000000  400.000000  400.000000   
mean       12.697500                      56.000208   51.804674    0.547500   
std       111.107417                      27.137584   10.948522    0.498362   
min         0.000000                       0.000000   23.000000    0.000000   
25%         0.000000                      38.000000   44.516769    0.000000   
50%         0.000000                      55.000000   51.019507    1.000000   
75%         0.000000                      73.000000   60.000000    1.000000   
max       999.000000                     144.000000   79.603012    1.000000   

              PgR        HER2  TrippleNegative  ChemoGrade  Proliferation  \
count  400.000000  400.000000       400.000000  400.000000     400.000000   
mean     2.902500    2.797500         2.830000    9.875000       6.562500   
std     49.932114   49.937068        49.935558   86.09291

In [6]:
df = df.rename(columns={'ID': 'PatientID', 'pCR (outcome)': 'PCR', 'RelapseFreeSurvival (outcome)': 'RFS'})

In [7]:
# # replace "999" with NaN (Not a Number)
df = df.replace(999, np.nan)
print(df['RFS'].isna().sum())

# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # # Assuming df is your loaded DataFrame
# # df_fixed = df.copy()

# # # List of columns to check for '999' as a missing value code
# # missing_cols = ['PgR', 'HER2', 'TrippleNegative', 'ChemoGrade', 'Proliferation', 
# #                 'HistologyType', 'LNStatus', 'Gene']

# # # Replace 999 with NaN (numpy.nan)
# # df_fixed[missing_cols] = df_fixed[missing_cols].replace(999, np.nan)

# # Now proceed with your existing train-test split:
# # X = df_fixed.drop(['ID', 'pCR (outcome)', 'RelapseFreeSurvival (outcome)'], axis=1)
# # y = df_fixed['RelapseFreeSurvival (outcome)']
# # X_train, X_test, y_train, y_test = train_test_split(...)

0


In [8]:
print(df.head())

   PatientID  PCR    RFS   Age  ER  PgR  HER2  TrippleNegative  ChemoGrade  \
0  TRG002174  1.0  144.0  41.0   0  0.0   0.0              1.0         3.0   
1  TRG002178  0.0  142.0  39.0   1  1.0   0.0              0.0         3.0   
2  TRG002204  1.0  135.0  31.0   0  0.0   0.0              1.0         2.0   
3  TRG002206  0.0   12.0  35.0   0  0.0   0.0              1.0         3.0   
4  TRG002210  0.0  109.0  61.0   1  0.0   0.0              0.0         2.0   

   Proliferation  ...  original_glszm_SmallAreaHighGrayLevelEmphasis  \
0            3.0  ...                                       0.517172   
1            3.0  ...                                       0.444391   
2            1.0  ...                                       0.534549   
3            3.0  ...                                       0.506185   
4            1.0  ...                                       0.462282   

   original_glszm_SmallAreaLowGrayLevelEmphasis  original_glszm_ZoneEntropy  \
0                  

In [9]:
for idx, col in enumerate(df.columns):
    print(idx, col)

0 PatientID
1 PCR
2 RFS
3 Age
4 ER
5 PgR
6 HER2
7 TrippleNegative
8 ChemoGrade
9 Proliferation
10 HistologyType
11 LNStatus
12 TumourStage
13 Gene
14 original_shape_Elongation
15 original_shape_Flatness
16 original_shape_LeastAxisLength
17 original_shape_MajorAxisLength
18 original_shape_Maximum2DDiameterColumn
19 original_shape_Maximum2DDiameterRow
20 original_shape_Maximum2DDiameterSlice
21 original_shape_Maximum3DDiameter
22 original_shape_MeshVolume
23 original_shape_MinorAxisLength
24 original_shape_Sphericity
25 original_shape_SurfaceArea
26 original_shape_SurfaceVolumeRatio
27 original_shape_VoxelVolume
28 original_firstorder_10Percentile
29 original_firstorder_90Percentile
30 original_firstorder_Energy
31 original_firstorder_Entropy
32 original_firstorder_InterquartileRange
33 original_firstorder_Kurtosis
34 original_firstorder_Maximum
35 original_firstorder_MeanAbsoluteDeviation
36 original_firstorder_Mean
37 original_firstorder_Median
38 original_firstorder_Minimum
39 origina

In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 121 entries, PatientID to original_ngtdm_Strength
dtypes: float64(117), int64(3), object(1)
memory usage: 378.3+ KB
None


In [11]:
#features with missing values (NaN)
print(df.drop(columns=['PatientID', 'PCR', 'RFS']).isna().sum().sort_values(ascending=False).head(15))

Gene                                                  88
ChemoGrade                                             3
HistologyType                                          3
Proliferation                                          2
PgR                                                    1
HER2                                                   1
TrippleNegative                                        1
LNStatus                                               1
original_glrlm_HighGrayLevelRunEmphasis                0
original_gldm_SmallDependenceHighGrayLevelEmphasis     0
original_gldm_SmallDependenceLowGrayLevelEmphasis      0
original_glrlm_GrayLevelNonUniformity                  0
original_glrlm_GrayLevelNonUniformityNormalized        0
original_glrlm_GrayLevelVariance                       0
Age                                                    0
dtype: int64


In [12]:
#defining feature and targets
y = df['RFS']
X = df.drop(columns=['PatientID', 'PCR', 'RFS'])
# X = df_fixed.drop(['PatientID', 'PCR', 'RFS'], axis=1)
# y = df_fixed['RFS']

mask = y.notna()
X = X[mask]
y = y[mask]

print(X.shape)
print(y.shape)
print(y.isna().sum())

(400, 118)
(400,)
0


# Preprocessing piplelines

In [13]:
# important features to be retained as per the assignment:ER, HER2 and Gene (Given in assignment: Implementation Requirement)

imp_features = ['ER', 'HER2', 'Gene']
# other_features = [col for col in X.columns if col not in imp_features]
mri_features = [col for col in X.columns if col.startswith('original_')]
other_features = [col for col in X.columns if col not in imp_features and col not in mri_features]

print("Total features: " + str(len(X.columns)))
print("Important features: " + str(len(imp_features)))
print("MRI features: " + str(len(mri_features)))
print("Other Clinical features: " + str(len(other_features)))

Total features: 118
Important features: 3
MRI features: 107
Other Clinical features: 8


# split train and test

In [14]:
# split train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(X_train.shape, X_test.shape)


(320, 118) (80, 118)


 # Building pipeline and PCA

In [15]:
from sklearn.compose import ColumnTransformer

# Preprocessing for important features
imp_transform_knn = Pipeline([('imputer', KNNImputer(n_neighbors=5)),('scaler', 'passthrough')])

# Preprocessing for MRI and other clinical features
mri_transform_knn = Pipeline([('imputer', KNNImputer(n_neighbors=5)),('scaler', StandardScaler()), ('pca_std', PCA(n_components=0.95))])
mri_transform_knn_pca = Pipeline([('imputer', KNNImputer(n_neighbors=5)),('scaler', RobustScaler()),('pca_robust',PCA(n_components=0.95, whiten=True))])

other_transform_knn = Pipeline([('imputer', KNNImputer(n_neighbors=5)),('scaler', StandardScaler())])
other_transform_knn_pca = Pipeline([('imputer', KNNImputer(n_neighbors=5)),('scaler', RobustScaler())])


# Combine preprocessing
preprocessor_knn = ColumnTransformer([
    ('imp', imp_transform_knn, imp_features),
    ('mri', mri_transform_knn, mri_features),
    ('other', other_transform_knn, other_features)
])

preprocessor_knn_pca = ColumnTransformer([
    ('imp', imp_transform_knn, imp_features),
    ('mri', mri_transform_knn_pca, mri_features),
    ('other', other_transform_knn_pca, other_features)
])


<hr>

In [16]:
imp_transform_rf = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', 'passthrough')
])

mri_transform_rf = Pipeline([
    # ('imputer', IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, random_state=42), initial_strategy='median', max_iter=3)),
    ('imputer', IterativeImputer(random_state=42, initial_strategy='median', max_iter=5)),
    ('scaler', StandardScaler()),
    ('pca_std', PCA(n_components=0.95))
])

mri_transform_rf_pca = Pipeline([
    # ('imputer',  IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, max_depth=5, random_state=42), initial_strategy='median', max_iter=3)),
    ('imputer', IterativeImputer(random_state=42, initial_strategy='median', max_iter=5)),
    ('scaler', RobustScaler()),
    ('pca_robust',PCA(n_components=0.95, whiten=True))
])

other_transform_rf = Pipeline([
    ('imputer', IterativeImputer(random_state=42, initial_strategy='median', max_iter=5)),
    # ('imputer', IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, random_state=42), initial_strategy='median', max_iter=5)),
    ('scaler', StandardScaler()),
])
other_transform_rf_pca = Pipeline([
    # ('imputer', IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, random_state=42), initial_strategy='median', max_iter=5)),
    ('imputer', IterativeImputer(random_state=42, initial_strategy='median', max_iter=5)),
    ('scaler', RobustScaler())
])

# Combine preprocessing
preprocessor_rf = ColumnTransformer([
    ('imp', imp_transform_rf, imp_features),
    ('mri', mri_transform_rf, mri_features),
    ('other', other_transform_rf, other_features)
])

preprocessor_rf_pca = ColumnTransformer([
    ('imp', imp_transform_rf, imp_features),
    ('mri', mri_transform_rf_pca, mri_features),
    ('other', other_transform_rf_pca, other_features)
])

# Models and hyperparameters

In [17]:
# Model and hyperparameters
from sklearn.model_selection import GridSearchCV

models = {
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "SVR": SVR(),
    "XGBoost": XGBRegressor(objective='reg:absoluteerror', n_jobs=-1, random_state=42),
}

param_grids_std = {
    "RandomForest": {
        'model__n_estimators': [100, 200],
        'model__max_depth': [5, 10, None],
        'preprocessor__mri__pca_std__n_components': [0.80, 0.90, 0.95, 0.99]
    },
    
    "GradientBoosting": {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1],
        'preprocessor__mri__pca_std__n_components': [0.80, 0.90, 0.95, 0.99]
    },
    
    "SVR": [
        # RBF Kernel
        {
            'model__kernel': ['rbf'],
            'model__C': [0.1, 1, 10, 100],
            'model__gamma': ['scale', 0.01, 0.1, 1],
            'model__epsilon': [0.01, 0.1, 0.5],
            'preprocessor__mri__pca_std__n_components': [0.80, 0.90, 0.95, 0.99]
        },
        # Linear Kernel
        {
            'model__kernel': ['linear'],
            'model__C': [0.1, 1, 10, 100],
            'model__epsilon': [0.01, 0.1, 0.5],
            'preprocessor__mri__pca_std__n_components': [0.80, 0.90, 0.95, 0.99]
        },

        #polynomial 
        {
            'model__kernel': ['poly'],
            'model__C': [5],
            'model__degree': [2, 3],
            'model__coef0': [1],
            'preprocessor__mri__pca_std__n_components': [0.80, 0.90, 0.95, 0.99]
        }
    ],
    
    "XGBoost": {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1],
        'model__max_depth': [3, 5],
        'model__reg_alpha': [0, 0.1, 1], # L1 regularization
        'preprocessor__mri__pca_std__n_components': [0.80, 0.90, 0.95, 0.99]
    }
}

<hr>

In [18]:
# Model and hyperparameters
from sklearn.model_selection import GridSearchCV

models_pca = {
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "SVR": SVR(),
    "XGBoost": XGBRegressor(objective='reg:absoluteerror', n_jobs=-1, random_state=42),
}

param_grids_robust = {
    "RandomForest": {
        'model__n_estimators': [100, 200],
        'model__max_depth': [5, 10, None],
        'preprocessor__mri__pca_robust__n_components': [0.80, 0.90, 0.95, 0.99]
    },
    
    "GradientBoosting": {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1],
        'preprocessor__mri__pca_robust__n_components': [0.80, 0.90, 0.95, 0.99]
    },
    
    "SVR": [
        # RBF Kernel
        {
            'model__kernel': ['rbf'],
            'model__C': [0.1, 1, 10, 100],
            'model__gamma': ['scale', 0.01, 0.1, 1],
            'model__epsilon': [0.01, 0.1, 0.5],
            'preprocessor__mri__pca_robust__n_components': [0.80, 0.90, 0.95, 0.99]
        },
        # Linear Kernel
        {
            'model__kernel': ['linear'],
            'model__C': [0.1, 1, 10, 100],
            'model__epsilon': [0.01, 0.1, 0.5],
            'preprocessor__mri__pca_robust__n_components': [0.80, 0.90, 0.95, 0.99]
        },
        #polynomial 
        {
            'model__kernel': ['poly'],
            'model__C': [5],
            'model__degree': [2, 3],
            'model__coef0': [1],
            'preprocessor__mri__pca_robust__n_components': [0.80, 0.90, 0.95, 0.99]
        }
    ],
    
    "XGBoost": {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1],
        'model__max_depth': [3, 5],
        'model__reg_alpha': [0, 0.1, 1], 
        'preprocessor__mri__pca_robust__n_components': [0.80, 0.90, 0.95, 0.99]
    }
}

# Model Training, Selection, and Final Test Evaluation

In [19]:
best_models_knn = {}
model_scores_knn = {}

for model_name, model in models.items():
    pipe = Pipeline([('preprocessor', preprocessor_knn),('model', model)])
    
    # Grid search
    gs = GridSearchCV(pipe, param_grid=param_grids_std[model_name],scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
    
    gs.fit(X_train, y_train)
    
    best_models_knn[model_name] = gs.best_estimator_
    model_scores_knn[model_name] = -gs.best_score_
    
    print(model_name + " best parameters: " + str(gs.best_params_))
    print(model_name + " best CV MAE: " + str(round(-gs.best_score_, 2)))


RandomForest best parameters: {'model__max_depth': 5, 'model__n_estimators': 100, 'preprocessor__mri__pca_std__n_components': 0.99}
RandomForest best CV MAE: 20.97
GradientBoosting best parameters: {'model__learning_rate': 0.01, 'model__n_estimators': 100, 'preprocessor__mri__pca_std__n_components': 0.99}
GradientBoosting best CV MAE: 20.96
SVR best parameters: {'model__C': 100, 'model__epsilon': 0.5, 'model__gamma': 0.1, 'model__kernel': 'rbf', 'preprocessor__mri__pca_std__n_components': 0.99}
SVR best CV MAE: 20.74
XGBoost best parameters: {'model__learning_rate': 0.01, 'model__max_depth': 5, 'model__n_estimators': 100, 'model__reg_alpha': 1, 'preprocessor__mri__pca_std__n_components': 0.8}
XGBoost best CV MAE: 20.84


In [20]:
best_models_knn_pca = {}
model_scores_knn_pca = {}

for model_name, model in models_pca.items():
    pipe = Pipeline([
        ('preprocessor', preprocessor_knn_pca),
        ('model', model)
    ])
    
    # Grid search
    gs = GridSearchCV(pipe, param_grid=param_grids_robust[model_name], 
                      scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
    
    gs.fit(X_train, y_train)
    
    best_models_knn_pca[model_name] = gs.best_estimator_
    model_scores_knn_pca[model_name] = -gs.best_score_
    
    print(model_name + " best parameters: " + str(gs.best_params_))
    print(model_name + " best CV MAE: " + str(round(-gs.best_score_, 2)))

RandomForest best parameters: {'model__max_depth': 5, 'model__n_estimators': 200, 'preprocessor__mri__pca_robust__n_components': 0.8}
RandomForest best CV MAE: 21.8
GradientBoosting best parameters: {'model__learning_rate': 0.01, 'model__n_estimators': 100, 'preprocessor__mri__pca_robust__n_components': 0.8}
GradientBoosting best CV MAE: 21.16
SVR best parameters: {'model__C': 1, 'model__epsilon': 0.1, 'model__gamma': 1, 'model__kernel': 'rbf', 'preprocessor__mri__pca_robust__n_components': 0.8}
SVR best CV MAE: 21.07
XGBoost best parameters: {'model__learning_rate': 0.01, 'model__max_depth': 5, 'model__n_estimators': 100, 'model__reg_alpha': 0.1, 'preprocessor__mri__pca_robust__n_components': 0.8}
XGBoost best CV MAE: 20.78


<hr>

In [21]:
best_models_rf = {}
model_scores_rf = {}

for model_name, model in models.items():
    pipe = Pipeline([
        ('preprocessor', preprocessor_rf),
        ('model', model)
    ])
    
    # Grid search
    gs = GridSearchCV(pipe, param_grid=param_grids_std[model_name], 
                      scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)
    
    gs.fit(X_train, y_train)
    best_models_rf[model_name] = gs.best_estimator_
    model_scores_rf[model_name] = -gs.best_score_
    
    print(model_name + " best parameters: " + str(gs.best_params_))
    print(model_name + " best CV MAE: " + str(round(-gs.best_score_, 2)))

RandomForest best parameters: {'model__max_depth': 5, 'model__n_estimators': 200, 'preprocessor__mri__pca_std__n_components': 0.99}
RandomForest best CV MAE: 20.75
GradientBoosting best parameters: {'model__learning_rate': 0.01, 'model__n_estimators': 100, 'preprocessor__mri__pca_std__n_components': 0.9}
GradientBoosting best CV MAE: 21.02
SVR best parameters: {'model__C': 100, 'model__epsilon': 0.5, 'model__gamma': 0.1, 'model__kernel': 'rbf', 'preprocessor__mri__pca_std__n_components': 0.99}
SVR best CV MAE: 20.71
XGBoost best parameters: {'model__learning_rate': 0.01, 'model__max_depth': 5, 'model__n_estimators': 200, 'model__reg_alpha': 1, 'preprocessor__mri__pca_std__n_components': 0.99}
XGBoost best CV MAE: 20.79


In [22]:
best_models_rf_pca = {}
model_scores_rf_pca = {}

for model_name, model in models_pca.items():
    pipe = Pipeline([
        ('preprocessor', preprocessor_rf_pca),
        ('model', model)
    ])
    
    # Grid search
    gs = GridSearchCV(pipe, param_grid=param_grids_robust[model_name], 
                      scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)
    
    gs.fit(X_train, y_train)
    
    best_models_rf_pca[model_name] = gs.best_estimator_
    model_scores_rf_pca[model_name] = -gs.best_score_
    
    print(model_name + " best parameters: " + str(gs.best_params_))
    print(model_name + " best CV MAE: " + str(round(-gs.best_score_, 2)))

RandomForest best parameters: {'model__max_depth': 5, 'model__n_estimators': 200, 'preprocessor__mri__pca_robust__n_components': 0.8}
RandomForest best CV MAE: 21.78
GradientBoosting best parameters: {'model__learning_rate': 0.01, 'model__n_estimators': 100, 'preprocessor__mri__pca_robust__n_components': 0.8}
GradientBoosting best CV MAE: 21.13
SVR best parameters: {'model__C': 1, 'model__epsilon': 0.01, 'model__gamma': 1, 'model__kernel': 'rbf', 'preprocessor__mri__pca_robust__n_components': 0.8}
SVR best CV MAE: 20.99
XGBoost best parameters: {'model__learning_rate': 0.01, 'model__max_depth': 3, 'model__n_estimators': 100, 'model__reg_alpha': 1, 'preprocessor__mri__pca_robust__n_components': 0.8}
XGBoost best CV MAE: 21.27


# Final Model selection

In [23]:
#final model selection
all_best_models = []

# KNN (Standard)
best_model_name = min(model_scores_knn, key=model_scores_knn.get)
best_model = best_models_knn[best_model_name]
# testing on the test set
best_pred = best_model.predict(X_test)
best_mae = mean_absolute_error(y_test, best_pred)
print("KNN Best model: " + str(best_model_name))
print("Test MAE: " + str(round(best_mae, 2)))
all_best_models.append(("KNN_" + str(best_model_name), best_model, best_mae))

KNN Best model: SVR
Test MAE: 22.21


In [24]:
# KNN (PCA)
best_model_name = min(model_scores_knn_pca, key=model_scores_knn_pca.get)
best_model = best_models_knn_pca[best_model_name]
best_pred = best_model.predict(X_test)
best_mae = mean_absolute_error(y_test, best_pred)
print("KNN (PCA) Best model: " + str(best_model_name))
print("Test MAE: " + str(round(best_mae, 2)))
all_best_models.append(("KNN_PCA_" + str(best_model_name), best_model, best_mae))

KNN (PCA) Best model: XGBoost
Test MAE: 21.55


In [25]:
# RF
best_model_name = min(model_scores_rf, key=model_scores_rf.get)
best_model = best_models_rf[best_model_name]
best_pred = best_model.predict(X_test)
best_mae = mean_absolute_error(y_test, best_pred)

print("RF Best model: " + str(best_model_name))
print("Test MAE: " + str(round(best_mae, 2)))
all_best_models.append(("RF_" + str(best_model_name), best_model, best_mae))

RF Best model: SVR
Test MAE: 22.2


In [26]:
# RF(PCA)
best_model_name = min(model_scores_rf_pca, key=model_scores_rf_pca.get)
best_model = best_models_rf_pca[best_model_name]
best_pred = best_model.predict(X_test)
best_mae = mean_absolute_error(y_test, best_pred)
print("RF PCA Best model: " + str(best_model_name))
print("Test MAE: " + str(round(best_mae, 2)))
all_best_models.append(("RF_PCA_" + str(best_model_name), best_model, best_mae))

RF PCA Best model: SVR
Test MAE: 21.46


In [27]:
# the model with the minimum MAE among all four 
final_mae = float('inf') 
final_model = None
final_model_name = ""

for name, model, mae in all_best_models:
    if mae < final_mae:
        final_mae = mae
        final_model_name = name
        final_model = model

print("the overall best model:" + final_model_name)
print("final test MAE:", round(final_mae, 2))

# Saving the FINAL model
joblib.dump(final_model, 'rfs_model.joblib')

the overall best model:RF_PCA_SVR
final test MAE: 21.46


['rfs_model.joblib']